# Visualising foursquare data and breaking it down into sectors

In [2]:
import json
import glob
import io
import os
import re
import itertools as iter
import numpy as np

# data
import pandas as pd
import shapely as shp
import geopandas as gpd
import matplotlib.pyplot as plt
import polars as pl
import geoarrow

# import shapely
import duckdb
import overturemaps
import h3
import scalenav
import scalenav.data as snd
import scalenav.scale_nav as sn
from scalenav.plotting import cmap
import scalenav.oop as snoo
import jinja2
from ipywidgets import HTML


# from scipy import io as io
# import nctoolkit as nc
# import xarray as xr
# import rioxarray as rx

import ibis as ib

from ibis import _

import ibis.selectors as s

ib.options.interactive = True
ib.options.graphviz_repr = True

# plots

from wordcloud import WordCloud

# from datashader import transfer_functions as tf, reductions as rd
import pypalettes as pypal
import pydeck as pdk
from seaborn import color_palette

Using angles for meter grid.
Using angles for meter grid.
Using angles for meter grid.
Using angles for meter grid.
Using angles for meter grid.


## Color palettes 

In [3]:
from parameters import *

# plot_limit = 10_000

# # continuous palette
# count_palette_name = "OrYel"

# count_pal = pypal.load_cmap(count_palette_name)

# # categorical palette
# cat_pal_name = "Bold"

# cat_pal = pypal.load_cmap(name=cat_pal_name)

# map_limits = [-168.8,-56.9,189.8,77.7]

## loading data 

In [ ]:
conn = snoo.sn_connect()

In [7]:
places = conn.read_parquet("/Users/cenv1069/Documents/data/datasets/foursquare/raw/places/*")

In [8]:
places.head(10)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ fsq_place_id             ┃ name                                     ┃ latitude  ┃ longitude   ┃ address               ┃ locality              ┃ region        ┃ postcode ┃ admin_region ┃ post_town ┃ po_box ┃ country ┃ date_created ┃ date_refreshed ┃ date_closed ┃ tel            ┃ website                         ┃ email                                   ┃ facebook_id      ┃ instagram ┃ twitter ┃ fsq_category_ids             ┃ fsq_category_labels                                                                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string                   │ string                                   │ float64   │ float64     │ string                │ string                │ string        │ string   │ string       │ string    │ string │ string  │ string       │ string         │ string      │ string         │ string                          │ string                                  │ int64            │ string    │ string  │ array<string>                │ array<string>                                                                          │
├──────────────────────────┼──────────────────────────────────────────┼───────────┼─────────────┼───────────────────────┼───────────────────────┼───────────────┼──────────┼──────────────┼───────────┼────────┼─────────┼──────────────┼────────────────┼─────────────┼────────────────┼─────────────────────────────────┼─────────────────────────────────────────┼──────────────────┼───────────┼─────────┼──────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│ 4aee4d4688a04abe82d1ace4 │ Canada House                             │ 41.781153 │    3.029574 │ Anselm Clavé, 16 Bajo │ Sant Feliu de Guíxols │ Gerona        │ 17220    │ Cataluña     │ NULL      │ NULL   │ ES      │ 2010-05-09   │ 2024-06-14     │ NULL        │ 972 32 45 54   │ ]8;id=647966;http://www.canadahouse.es\http://www.canadahouse.es]8;;\       │ NULL                                    │             NULL │ NULL      │ NULL    │ ['4bf58dd8d48988d103951735'] │ ['Retail > Fashion Retail > Clothing Store']                                           │
│ cb57d89eed29405b908b0b6e │ Fotos. Zakład fotograficzny. Głowacka T. │ 52.192667 │   18.633436 │ Mickiewicza 8         │ Koło                  │ Wielkopolskie │ 62-600   │ NULL         │ NULL      │ NULL   │ PL      │ 2015-05-06   │ 2024-06-27     │ NULL        │ 63 272 08 68   │ NULL                            │ teresa.glowacka.fotos.kolo@neostrada.pl │             NULL │ NULL      │ NULL    │ ['4d4b7105d754a06378d81259'] │ ['Retail']                                                                             │
│ 59a4553d112c6c2b6c378e08 │ CoHo                                     │ 40.774559 │  -73.871849 │ NULL                  │ New York              │ NY            │ 11371    │ NULL         │ NULL      │ NULL   │ US      │ 2017-08-28   │ 2024-10-25     │ 2019-08-22  │ NULL           │ NULL                            │ NUL

In [9]:
places = places.filter(
    _.longitude>map_limits[0],
    _.latitude>map_limits[1],
    _.longitude<map_limits[2],
    _.latitude<map_limits[3],
    )

In [10]:
places.columns

['fsq_place_id',
 'name',
 'latitude',
 'longitude',
 'address',
 'locality',
 'region',
 'postcode',
 'admin_region',
 'post_town',
 'po_box',
 'country',
 'date_created',
 'date_refreshed',
 'date_closed',
 'tel',
 'website',
 'email',
 'facebook_id',
 'instagram',
 'twitter',
 'fsq_category_ids',
 'fsq_category_labels']

In [11]:
places = places.select("fsq_place_id","name","latitude","longitude","locality","region","country","fsq_category_ids","fsq_category_labels")

In [12]:
# ib.to_sql(places)

In [13]:
h3_res = 5

In [ ]:
places_h3 = snoo.h3_project(places,res=h3_res)

# 
# places.alias("b").sql(f"""
# Select *, h3_h3_to_string(h3_latlng_to_cell(latitude,longitude,{h3_res})) as h3_id
# from b;
# """)

places_h3.head()

Assuming coordinates columns ('longitude','latitude')


┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ fsq_place_id             ┃ name                                     ┃ latitude  ┃ longitude   ┃ locality              ┃ region        ┃ country ┃ fsq_category_ids             ┃ fsq_category_labels                                                                  ┃ h3_id           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string                   │ string                                   │ float64   │ float64     │ string                │ string        │ string  │ array<string>                │ array<string>                                                                        │ string          │
├──────────────────────────┼──────────────────────────────────────────┼───────────┼─────────────┼───────────────────────┼───────────────┼─────────┼──────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┼─────────────────┤
│ 4aee4d4688a04abe82d1ace4 │ Canada House                             │ 41.781153 │    3.029574 │ Sant Feliu de Guíxols │ Gerona        │ ES      │ ['4bf58dd8d48988d103951735'] │ ['Retail > Fashion Retail > Clothing Store']                                         │ 85394557fffffff │
│ cb57d89eed29405b908b0b6e │ Fotos. Zakład fotograficzny. Głowacka T. │ 52.192667 │   18.633436 │ Koło                  │ Wielkopolskie │ PL      │ ['4d4b7105d754a06378d81259'] │ ['Retail']                                                                           │ 851e253bfffffff │
│ 59a4553d112c6c2b6c378e08 │ CoHo                                     │ 40.774559 │  -73.871849 │ New York              │ NY            │ US      │ ['4bf58dd8d48988d110941735'] │ ['Dining and Drinking > Restaurant > Italian Restaurant']                            │ 852a100ffffffff │
│ 4bea3677415e20a110d8e4bb │ Bisma lounge                             │ -6.134262 │  106.864683 │ Sunter city           │ Jakarta utara │ ID      │ NULL                         │ NULL                                                                                 │ 858c12b7fffffff │
│ dca3aeba404e4b6006620a46 │ Grace Motorworks                         │ 34.180992 │ -118.537669 │ Tarzana               │ CA            │ US      │ ['52f2ab2ebcbc57f1066b8b44'] │ ['Business and Professional Services > Automotive Service > Automotive Repair Shop'] │ 8529a18bfffffff │
└──────────────────────────┴──────────────────────────────────────────┴───────────┴─────────────┴───────────────────────┴───────────────┴─────────┴──────────────────────────────┴──────────────────────────────────────────────────────────────────────────────────────┴─────────────────┘

In [15]:
# ib.to_sql(places_h3)

In [16]:
places_h3.select("h3_id").nunique()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────┐
│ 238100 │
└────────┘

## Closer inspection

In [17]:
closer_lookups = {
    "centre" : [-3,-3,3,3],
    "caribean" : [-85.85,17.58,-79.72,21.35],
    "caribean2" : [-76.86,21.45,-71.26,26.14],
}

lims = closer_lookups["centre"]

In [18]:
places_lookup = places_h3.filter(
    _.longitude>lims[0],
    _.latitude>lims[1],
    _.longitude<lims[2],
    _.latitude<lims[3],
    )

In [19]:
places_lookup.count()

┌─────┐
│ 880 │
└─────┘

In [20]:
places_lookup

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ fsq_place_id             ┃ name                       ┃ latitude  ┃ longitude ┃ locality ┃ region  ┃ country ┃ fsq_category_ids             ┃ fsq_category_labels                                                                    ┃ h3_id           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string                   │ string                     │ float64   │ float64   │ string   │ string  │ string  │ array<string>                │ array<string>                                                                          │ string          │
├──────────────────────────┼────────────────────────────┼───────────┼───────────┼──────────┼─────────┼─────────┼──────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────┼─────────────────┤
│ 57dd92a8498e958c99e32be5 │ İncirhan Han               │  0.839424 │ -1.966077 │ NULL     │ NULL    │ NULL    │ ['4bf58dd8d48988d116941735'] │ ['Dining and Drinking > Bar']                                                          │ 85755a93fffffff │
│ 576fe371498efee0076e17b5 │ Nieuwe Hangar              │  2.709495 │  0.230237 │ NULL     │ NULL    │ NULL    │ ['4bf58dd8d48988d164941735'] │ ['Landmarks and Outdoors > Plaza']                                                     │ 85754633fffffff │
│ 574b2d5038fa943557805447 │ 06 hc 7138                 │  1.867243 │  1.183678 │ NULL     │ NULL    │ NULL    │ ['4f2a210c4b9023bd5841ed28'] │ ['Community and Government > Housing Development']                                     │ 85754577fffffff │
│ 5757a67c498efcdd2cd767e5 │ Ahmet  Sevinç              │  0.585319 │  2.927339 │ İstanbul │ üsküdar │ NULL    │ ['4bf58dd8d48988d121941735'] │ ['Dining and Drinking > Bar > Lounge']                                                 │ 857569c3fffffff │
│ 57347522cd10494d75c7c206 │ Özdemir Makina İmalat      │  1.141297 │ -1.136648 │ Izmir    │ NULL    │ NULL    │ ['4eb1bea83b7b6f98df247e06'] │ ['Business and Professional Services > Factory']                                       │ 85755c73fffffff │
│ 57232a04498e11c97d2fbcf0 │ Samantas Hoods             │  0.019262 │  1.300770 │ NULL     │ NULL    │ NULL    │ ['5032891291d4c4b30a586d68'] │ ['Community and Government > Assisted Living']                                         │ 85754c37fffffff │
│ 593dd6d62ebb34774fa82391 │ Lugar de encuentro         │ -2.905220 │ -2.598113 │ NULL     │ NULL    │ NULL    │ ['4bf58dd8d48988d162941735'] │ ['Landmarks and Outdoors > Other Great Outdoors']                                      │ 8574c523fffffff │
│ 57e5443f498e418f6adc7e54 │ English Course             │  2.399225 │ -1.797922 │ NULL     │ NULL    │ NULL    │ ['4bf58dd8d48988d198941735'] │ ['Community and Government > Education > College and University > College Academic'+9] │ 85755383fffffff │
│ 576c071638fa0e98789447d9 │ Niğde  Ulukışla Horoz Köyü │ -1.557929 │ -1.539377 │ NULL     │ NULL    │ NULL    │ ['4f2a210c4b9023bd5841ed28'] │ ['Community and Government > Housing Development']                                     │ 8574e893fffffff │
│ 5c7e93d6d4cc98002c7a1ee7 │ Europen                    │ -2.454680 │ -1.372820 │ NULL     │ NULL    │ NULL    │ ['4eb1bea83b7b6f98df247e06'] │ ['Business and Professional Services > Factory']                                       │ 8574e867fffffff │
│ …                        │ …                          │         … │         … │ …        │ …       │ …       │ …                            │ …                                                                                      │ … 

In [ ]:
# lookup_df = places_lookup.execute()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [22]:
# gpd.GeoDataFrame(
#     lookup_df,
#     geometry=gpd.GeoSeries(gpd.points_from_xy(lookup_df["longitude"],lookup_df["latitude"],crs="epsg:4326")),
# ).explore()

## Places categories

Expanding categories

In [23]:
places_h3 = (
    places_h3
    .mutate(cats=_.fsq_category_labels[0].split(" > "))
    # .mutate(cats_break=_.cats.split(" > "))
    # .select("cats","cats_break")
    .mutate(
        primary=_.cats[0],
        sec=_.cats[1],
        raw=_.cats[2],
            )
    .drop("cats")
)

In [24]:
places_h3.head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ fsq_place_id             ┃ name                                     ┃ latitude  ┃ longitude   ┃ locality              ┃ region        ┃ country ┃ fsq_category_ids             ┃ fsq_category_labels                                                                  ┃ h3_id           ┃ primary                            ┃ sec                ┃ raw                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string                   │ string                                   │ float64   │ float64     │ string                │ string        │ string  │ array<string>                │ array<string>                                                                        │ string          │ string                             │ string             │ string                 │
├──────────────────────────┼──────────────────────────────────────────┼───────────┼─────────────┼───────────────────────┼───────────────┼─────────┼──────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┼─────────────────┼────────────────────────────────────┼────────────────────┼────────────────────────┤
│ 4aee4d4688a04abe82d1ace4 │ Canada House                             │ 41.781153 │    3.029574 │ Sant Feliu de Guíxols │ Gerona        │ ES      │ ['4bf58dd8d48988d103951735'] │ ['Retail > Fashion Retail > Clothing Store']                                         │ 85394557fffffff │ Retail                             │ Fashion Retail     │ Clothing Store         │
│ cb57d89eed29405b908b0b6e │ Fotos. Zakład fotograficzny. Głowacka T. │ 52.192667 │   18.633436 │ Koło                  │ Wielkopolskie │ PL      │ ['4d4b7105d754a06378d81259'] │ ['Retail']                                                                           │ 851e253bfffffff │ Retail                             │ NULL               │ NULL                   │
│ 59a4553d112c6c2b6c378e08 │ CoHo                                     │ 40.774559 │  -73.871849 │ New York              │ NY            │ US      │ ['4bf58dd8d48988d110941735'] │ ['Dining and Drinking > Restaurant > Italian Restaurant']                            │ 852a100ffffffff │ Dining and Drinking                │ Restaurant         │ Italian Restaurant     │
│ 4bea3677415e20a110d8e4bb │ Bisma lounge                             │ -6.134262 │  106.864683 │ Sunter city           │ Jakarta utara │ ID      │ NULL                         │ NULL                                                                                 │ 858c12b7fffffff │ NULL                               │ NULL               │ NULL                   │
│ dca3aeba404e4b6006620a46 │ Grace Motorworks                         │ 34.180992 │ -118.537669 │ Tarzana               │ CA            │ US      │ ['52f2ab2ebcbc57f1066b8b44'] │ ['Business and Professional Services > Automotive Service > Automotive Repair Shop'] │ 8529a18bfffffff │ Business and Professional Services │ Automotive Service │ Automotive Repair Shop │
└──────────────────────────┴──────────────────────────────────────────┴───────────┴─────────────┴───────────────────────┴───────────────┴─────────┴──────────────────────────────┴──────────────────────────────────────────────────────────────────────────────────────┴─────────────────┴────────────────────────────────────┴──────────

In [25]:
places_h3.primary.value_counts().execute()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,primary,primary_count
0,Retail,15191632
1,Community and Government,11618950
2,Event,798664
3,Landmarks and Outdoors,6706523
4,Arts and Entertainment,3228616
5,Sports and Recreation,2214536
6,Travel and Transportation,7340595
7,Health and Medicine,4615604
8,Dining and Drinking,17277887
9,None,11525326


In [26]:
places_h3.sec.nunique()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────┐
│ 432 │
└─────┘

In [27]:
places_h3.sec.value_counts()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ sec                      ┃ sec_count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string                   │ int64     │
├──────────────────────────┼───────────┤
│ Furniture and Home Store │    812569 │
│ Office                   │   3620432 │
│ Stadium                  │    125276 │
│ Beach                    │    221447 │
│ Night Club               │    305991 │
│ Plaza                    │    157951 │
│ Museum                   │    142498 │
│ Office Supply Store      │    120977 │
│ Insurance Agency         │    373999 │
│ Lake                     │    141896 │
│ …                        │         … │
└──────────────────────────┴───────────┘

In [28]:
places_h3.raw.nunique()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────┐
│ 464 │
└─────┘

In [29]:
places_h3.raw.value_counts()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ raw                  ┃ raw_count ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string               │ int64     │
├──────────────────────┼───────────┤
│ Pizzeria             │    588357 │
│ Pet Grooming Service │     32593 │
│ Cheese Store         │     22789 │
│ Capitol Building     │     47357 │
│ Car Dealership       │    476951 │
│ General Contractor   │    574628 │
│ Town                 │     59672 │
│ Buddhist Temple      │     55406 │
│ Bathroom Contractor  │     31972 │
│ Home Service         │    280848 │
│ …                    │         … │
└──────────────────────┴───────────┘

In [30]:
# 
places_h3.filter(_.primary.isnull()).sec.isnull().all()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────┐
│ True │
└──────┘

In [31]:
places_h3.filter(_.sec.isnull()).primary.isnull().all()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┐
│ False │
└───────┘

In [32]:
places_h3.filter(_.sec.isnull()).raw.isnull().all()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────┐
│ True │
└──────┘

In [33]:
places_h3.filter(_.raw.isnull()).sec.isnull().all()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┐
│ False │
└───────┘

### Narrowing down categories

In [34]:
primary_exclude = ["Health and Medicine","Landmarks and Outdoors","Community and Government"]

In [35]:
places_h3 = places_h3.filter(~_.primary.isin(primary_exclude))

In [36]:
print(places_h3.count())
places_h3.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────┐
│ 69709790 │
└──────────┘


┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ fsq_place_id             ┃ name                                     ┃ latitude  ┃ longitude   ┃ locality              ┃ region        ┃ country ┃ fsq_category_ids             ┃ fsq_category_labels                                                                    ┃ h3_id           ┃ primary                            ┃ sec                ┃ raw                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string                   │ string                                   │ float64   │ float64     │ string                │ string        │ string  │ array<string>                │ array<string>                                                                          │ string          │ string                             │ string             │ string                 │
├──────────────────────────┼──────────────────────────────────────────┼───────────┼─────────────┼───────────────────────┼───────────────┼─────────┼──────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────┼─────────────────┼────────────────────────────────────┼────────────────────┼────────────────────────┤
│ 4aee4d4688a04abe82d1ace4 │ Canada House                             │ 41.781153 │    3.029574 │ Sant Feliu de Guíxols │ Gerona        │ ES      │ ['4bf58dd8d48988d103951735'] │ ['Retail > Fashion Retail > Clothing Store']                                           │ 85394557fffffff │ Retail                             │ Fashion Retail     │ Clothing Store         │
│ cb57d89eed29405b908b0b6e │ Fotos. Zakład fotograficzny. Głowacka T. │ 52.192667 │   18.633436 │ Koło                  │ Wielkopolskie │ PL      │ ['4d4b7105d754a06378d81259'] │ ['Retail']                                                                             │ 851e253bfffffff │ Retail                             │ NULL               │ NULL                   │
│ 59a4553d112c6c2b6c378e08 │ CoHo                                     │ 40.774559 │  -73.871849 │ New York              │ NY            │ US      │ ['4bf58dd8d48988d110941735'] │ ['Dining and Drinking > Restaurant > Italian Restaurant']                              │ 852a100ffffffff │ Dining and Drinking                │ Restaurant         │ Italian Restaurant     │
│ dca3aeba404e4b6006620a46 │ Grace Motorworks                         │ 34.180992 │ -118.537669 │ Tarzana               │ CA            │ US      │ ['52f2ab2ebcbc57f1066b8b44'] │ ['Business and Professional Services > Automotive Service > Automotive Repair Shop']   │ 8529a18bfffffff │ Business and Professional Services │ Automotive Service │ Automotive Repair Shop │
│ 505a6a8be4b066ff885b3204 │ 沖縄海邦銀行 国頭支店                    │ 26.732990 │  128.203686 │ 国頭郡国頭村          │ 沖縄県        │ JP      │ ['4bf58dd8d48988d10a951735'] │ ['Business and Professional Services > Financial Service > Banking and Finance > B'+3] │ 854a26b7fffffff │ Business and Professional Services │ Financial Service  │ Banking and Finance    │
└──────────────────────────┴──────────────────────────────────────────┴───────────┴─────────────┴───────────────────────┴───────────────┴─────────┴──────────────────────────────┴────────────────────────────────────────────────────────────────────────────────────────┴─────────────────┴────────────────────────────────────┴───────

In [ ]:
fsq_processed_file = "../datasets/foursquare/processed/places.parquet"
if not os.path.exists(fsq_processed_file):
    places_h3.select("fsq_place_id","name","latitude","longitude","locality","region","country","primary","sec","raw").to_parquet(fsq_processed_file)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Primary categories groups

In [36]:
# places_h3.group_by("h3_id").mutate(count=_.count())

## Places density

In [37]:
places_count = places_h3.h3_id.value_counts()

In [38]:
places_count.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ h3_id           ┃ h3_id_count ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string          │ int64       │
├─────────────────┼─────────────┤
│ 85394557fffffff │        3674 │
│ 8529a18bfffffff │       45743 │
│ 85c2ec63fffffff │          36 │
│ 85ad3617fffffff │       12142 │
│ 851fa457fffffff │       18073 │
└─────────────────┴─────────────┘

### Low density categories

In [40]:
# ib.to_sql(places_count)
places_h3 = places_h3.join(places_count,"h3_id",how="left",)

In [41]:
low_dens_places_df = places_h3.filter(_.h3_id_count<4).execute()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [42]:
low_dens_places_df

,fsq_place_id,name,latitude,longitude,locality,region,country,fsq_category_ids,fsq_category_labels,h3_id,primary,sec,raw,h3_id_right,h3_id_count
0,5577e639498e93b583766fa5,Льонок,51.330140,31.677754,Куликовка,None,UA,[4bf58dd8d48988d1f9941735],[Retail > Food and Beverage Retail],851e61b3fffffff,Retail,Food and Beverage Retail,None,851e61b3fffffff,1
1,1575a6780cba4f0b1bb3e7e6,Resto Au Soleil Couchant,48.728138,-79.462230,Clerval,QC,CA,[4bf58dd8d48988d147941735],[Dining and Drinking > Restaurant > Diner],852b94d3fffffff,Dining and Drinking,Restaurant,Diner,852b94d3fffffff,2
2,640a2ccd29068136a728990b,Pharmacie Kantakare,16.280230,-16.136066,Ross Béthio,Région de Saint Louis,SN,[4bf58dd8d48988d10f951735],[Retail > Pharmacy],85541023fffffff,Retail,Pharmacy,None,85541023fffffff,3
3,4f82ec49e4b0e160bb920d36,Belén De Los Andaquíes,1.411036,-75.871689,None,None,CO,[4d4b7105d754a06379d81259],[Travel and Transportation],8566c043fffffff,Travel and Transportation,None,None,8566c043fffffff,3
4,5735ed06498e851ad9f2ebf8,El Negocio de Mamá,7.121814,0.012703,None,None,GH,[52f2ab2ebcbc57f1066b8b1d],[Business and Professional Services > Health a...,85752d1bfffffff,Business and Professional Services,Health and Beauty Service,Dry Cleaner,85752d1bfffffff,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120339,5531f8a7498ee29a12707829,Hero Horce Riding Club,39.112548,116.926876,None,None,CN,[52e81612bcbc57f1066b7a2e],[Sports and Recreation > Sports Club],85318c47fffffff,Sports and Recreation,Sports Club,None,85318c47fffffff,3
120340,53fc8d9c498e738553d412b2,Posto Oeste Bahia,-12.925580,-45.974480,None,None,BR,[52e81612bcbc57f1066b79f4],[Dining and Drinking > Restaurant > Buffet],858129c3fffffff,Dining and Drinking,Restaurant,Buffet,858129c3fffffff,3
120341,53455551498eeaed8f6872b9,Сбербанк,48.198031,136.124949,поселок городского типа Мухен,None,RU,[4bf58dd8d48988d10a951735],[Business and Professional Services > Financia...,8514d2cffffffff,Business and Professional Services,Financial Service,Banking and Finance,8514d2cffffffff,2
120342,56a632ce498e6693802ae023,comercial BA Lucena,-6.654443,-64.291641,None,None,BR,[4eb1bea83b7b6f98df247e06],[Business and Professional Services > Factory],858a0ccffffffff,Business and Professional Services,Factory,None,858a0ccffffffff,1


In [ ]:
dens_thres = 2

In [ ]:
places_count_df = places_count.filter(_.h3_id_count>dens_thres).execute()

In [ ]:
places_count_df.head()

## Plotting 

In [ ]:
places_count_df["cols"] = cmap(places_count_df["h3_id_count"],palette=count_pal,log=True)

In [ ]:

# Define a layer to display on a map
layer = pdk.Layer(
    "H3HexagonLayer",
    places_count_df,#.sample(300_000),
    pickable=True,
    stroked=True,
    filled=True,
    opacity = .6,
    extruded=False,
    get_hexagon="h3_id",
    get_fill_color="cols",
    get_line_color=[255, 255, 255,0],
    line_width_min_pixels=0,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=0, longitude=0, zoom=3, bearing=0, pitch=30)


# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "Count: {h3_id_count}"})
r.to_html("../deck_maps/foursquare_places_dens_layer.html",
          iframe_height=700,
          )